The initial  template for this code was provided by the professor Giovanni Squillero `<giovanni.squillero@polito.it>`. All copyrights **`(c)`** are reserved to him. \
The original code can be found at [`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  


In [441]:
import logging
from itertools import combinations,permutations
from dataclasses import dataclass
import pandas as pd
import numpy as np
import geopy.distance
import random
import copy
from tqdm import tqdm
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

## Lab2 - TSP



In [442]:
cities = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
cities.head()

N_STEPS = 10_000

In [443]:
dist_matrix = np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geopy.distance.geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

In [444]:
@dataclass #decorator that generates some methods. Used to create a class with attributes
class Individual:
    genome:list[int]
    fitness:float=None

def fitness(individual:Individual)->int: #cost function to minimize. It calculates the total distance of the path
    tot_cost = 0

    for c1, c2 in zip(individual.genome, individual.genome[1:]):
        tot_cost += dist_matrix[c1, c2]
    #add the distance from the last city to the first
    tot_cost += dist_matrix[individual.genome[-1], individual.genome[0]]
    return tot_cost

def create_population(individuals_n:int, cities_n:int)->list[Individual]:
    population = []
    for _ in range(individuals_n):
        genome = list(range(1,cities_n)) #first city is always the same
        random.shuffle(genome)
        genome=[0]+genome #add the first city
        population.append(Individual(genome))
        for elem in population:
            elem.fitness = fitness(elem)
    return population

def tournament_uparent_selection(population:list[Individual]):
    #tournament challengers
    #generate two random numbers and select the individuals with the corresponding indexes
    idx1, idx2 = np.random.choice(len(population), size=2, replace=False)
    # ic(population[idx1].fitness,population[idx2].fitness)
    population.remove(population[idx1] if population[idx1].fitness > population[idx2].fitness else population[idx2])

def rank_based_parent_selection(population:list[Individual]):
    population.sort(key=lambda individual: individual.fitness) # Already sorted
    ranks = list(range(1, len(population) + 1))  # Rank 1 = best individual
    # probabilities = [1/r for r in ranks]  #Probabilities are inversely proportional to the rank
    #Normalize probabilities
    total = sum(ranks)
    probabilities = [p/total for p in ranks]
    selected_idx = np.random.choice(len(population), p=probabilities)
    population.pop(selected_idx)
    
  

def crossover2P(parent1:Individual, parent2:Individual)->Individual:
    offspring=[]
    start=np.random.randint(0,len(parent1.genome)-1)
    end=np.random.randint(start,len(parent1.genome))
    sub_path=parent1.genome[start:end]
    out_of_sub_path=[x for x in parent2.genome if x not in sub_path]
    for i in range(0,len(parent1.genome)):
        if(i>=start and i<end):
            offspring.append(sub_path.pop(0))
        else:
            offspring.append(out_of_sub_path.pop(0))
    return Individual(offspring)

def crossoverPopulation(population:list[Individual])->list[Individual]: #2 Point Crossover over the entire population
    offsprings=[]
    for i in range(len(population)//2):
        parent1,parent2=population[i],population[i+1]
        offsprings.append(crossover2P(parent1,parent2))
        offsprings.append(crossover2P(parent2,parent1))
  
    return offsprings

def mutationPopulation(population:list[Individual])->list[Individual]: #swap mutation over the entire population
    offspring=copy.deepcopy(population)
    for i in range (len(population)):
        modified_flag=False
        for _ in range (len(cities)): 
            if np.random.random()<1/len(offspring[i].genome): #mutation rate
                idx_city1 = np.random.randint(len(cities))
                idx_city2 = np.random.randint(len(cities))
                tmp = offspring[i].genome[idx_city1]
                offspring[i].genome[idx_city1] = offspring[i].genome[idx_city2]
                offspring[i].genome[idx_city2]=tmp
                modified_flag=True
        if modified_flag:
            offspring[i].fitness=fitness(offspring[i])
    return offspring

# def crossover(parent1: Individual, parent2: Individual) -> Individual:
#     cut_point = np.random.randint(len(parent1.genome))

#     offspring = [None] * len(parent1.genome)
    
#     genome_p1 = parent1.genome[:cut_point]
#     offspring[:cut_point] = genome_p1
    
#     genome_p2 = parent2.genome
#     fill_idx = cut_point
    
#     for gene in genome_p2:
#         if gene not in offspring:
#             offspring[fill_idx] = gene
#             fill_idx += 1
    
#     offspring = [gene for gene in offspring if gene is not None]
#     offspring.append(offspring[0])

    
#     return Individual(offspring)

def PMX_crossover(parent1:Individual, parent2:Individual)->tuple[Individual,Individual]:
    cut_1, cut_2 = sorted(np.random.choice(len(cities), size=2, replace=False))

    offspring1 = [None] * len(cities)
    offspring2 = [None] * len(cities)

    genome_p1 = parent1.genome.copy()[:-1]
    genome_p2 = parent2.genome.copy()[:-1]

    offspring1[cut_1:cut_2] = genome_p2[cut_1:cut_2]
    offspring2[cut_1:cut_2] = genome_p1[cut_1:cut_2]

    boundary_idx = list(range(0,cut_1)) + list(range(cut_2,len(cities)))
    
    for i in boundary_idx:
        if genome_p1[i] not in offspring1:
            offspring1[i] = genome_p1[i]
        else:
            tmp = genome_p1[i]
            j = i
            while tmp in offspring1:
                j = offspring1.index(genome_p1[j])
                tmp = offspring2[j]
            offspring1[i] = tmp

    for i in boundary_idx:
        if genome_p2[i] not in offspring2:
            offspring2[i] = genome_p2[i]
        else:
            tmp = genome_p2[i]
            j = i
            while tmp in offspring2:
                j = offspring2.index(genome_p2[j])
                tmp = offspring1[j]
            offspring2[i] = tmp
    
    offspring1 = [gene for gene in offspring1 if gene is not None]
    offspring2 = [gene for gene in offspring2 if gene is not None]
    offspring1.append(offspring1[0])
    offspring2.append(offspring2[0])
    return Individual(offspring1), Individual(offspring2)








In [445]:
def get_starting_solution()->Individual:
    """
    Compute a greedy sub-optimal solution
    """
    visited = np.full(len(cities), False)
    dist = dist_matrix.copy()
    city = 0
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    a=0

    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        # logging.debug(
        #     f"step: {cities.at[city,'name']} -> {cities.at[closest,'name']} ({dist_matrix[city,closest]:.2f}km)"
        # )
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    # logging.debug(
    #     f"step: {cities.at[tsp[-1],'name']} -> {cities.at[tsp[0],'name']} ({dist_matrix[tsp[-1],tsp[0]]:.2f}km)"
    # )
    tsp.append(tsp[0])
    return Individual(tsp)

def generate_solution_from_existing(parent:Individual)->Individual:
    new_genome = random.sample(parent.genome[:-1],len(parent.genome[:-1]))
    new_genome.append(new_genome[0])
    return Individual(new_genome)

In [446]:
# e = get_starting_solution()
# e2 = generate_solution_from_existing(e)
# offspring1= crossover(e, e2)
# # print(e.genome)
# # print(e2.genome)
# assert len(offspring1.genome[:-1]) == len(set(offspring1.genome))


In [447]:
# # We start from an already sub-optimal greedy solution and then we try to optimize it further
# starting_individual = get_starting_solution()
# tot_cost = fitness(starting_individual)
# starting_individual.fitness=tot_cost
# ic(tot_cost)

# OFFSPRING_SIZE= 5
# POPULATION_SIZE = 20

# # generate population from mutating the original father
# # population = [mutation(starting_individual) for _ in range(POPULATION_SIZE)]
# population = [generate_solution_from_existing(starting_individual) for _ in range(POPULATION_SIZE-1)]
# population.append(starting_individual)

# for elem in population:
#     elem.fitness = fitness(elem)

# for _ in tqdm(range(N_STEPS)):
#     offspring = list()
#     for _ in range(OFFSPRING_SIZE):
#         if np.random.random()<0.5:
#             p = parent_selection(population)
#             p2 = parent_selection(population)
#             o1= crossover(p,p2)
#             offspring.append(o1)

#         else:
#             p = parent_selection(population)
#             o = mutation(p,True)
#             offspring.append(o)
#     # random chance of producing a stochastic newborn
#     if  np.random.random()<0.5:
#         o = Individual(random.sample(starting_individual.genome,len(starting_individual.genome)))
#         offspring.append(o)
#     for i in offspring:
#         i.fitness = fitness(i)
#     population.extend(offspring)
#     population.sort(key=lambda i: i.fitness)
#     population = population[:POPULATION_SIZE]
#     # for e in population:
        
#     #     print(e.genome)
# for e in population:
#     print(e)
# ic(population[0].fitness)
# # logging.info(f"result: Found a path of {len(starting_individual)-1} steps, total length {tot_cost:.2f}km")

### Evolutionary Algorithm

In [448]:
# OFFSPRING_SIZE= 5
POPULATION_SIZE = 45
population=create_population(POPULATION_SIZE, len(cities))
start=get_starting_solution()
start.fitness=fitness(start)
start.genome=start.genome[:-1]
population.append(start)
# population.sort(key=lambda i: i.fitness)
# ic("Initial population:",population)
for _ in tqdm(range(100_000)):
    while len(population)>(POPULATION_SIZE+1)//2:
        rank_based_parent_selection(population)
    # ic("After parent selection:",population)
    # for _ in OFFSPRING_SIZE:
    if np.random.random()<0.1:
        offspring = crossoverPopulation(population)
        for o in offspring:
            o.fitness = fitness(o)
        population.extend(offspring)
    else:
        offspring=mutationPopulation(population) 
        population.extend(offspring)
        # ic("After mutation:",population)
  
    # ic("After crossover:",population)
    population.sort(key=lambda i: i.fitness)
    population = population[:POPULATION_SIZE]
    # ic("After sorting:",population)
ic("Best solution:",population[0].fitness)


100%|██████████| 100000/100000 [01:36<00:00, 1039.92it/s]
ic| "Best solution:": 'Best solution:'
    population[0].fitness: np.float64(4348.306038400042)


('Best solution:', np.float64(4348.306038400042))